In [29]:
import numpy as np
import yfinance as yf
import scipy.stats as stats
import pandas as pd
from math import pi

In [88]:
# symbol = 'EURUSD=X'

# # Fetch the historical data from 2014 to 2024
# data = yf.download(symbol, start='2014-01-01', end='2025-01-01', interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [6]:
data = pd.read_csv('usd_eur_ts.csv')
data.set_index('Date', inplace=True)
data.drop('Log_Returns', axis=1, inplace=True)

In [7]:
# data.to_csv('usd_eur_ts.csv', index=True)

In [8]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,1.376595,1.377467,1.363271,1.376671,1.376671,0
2014-01-03,1.366624,1.367297,1.360170,1.366662,1.366662,0
2014-01-06,1.359582,1.364610,1.357279,1.359601,1.359601,0
2014-01-07,1.363066,1.365799,1.359878,1.363196,1.363196,0
2014-01-08,1.361693,1.363605,1.357202,1.361637,1.361637,0
...,...,...,...,...,...,...
2024-05-20,1.087548,1.088495,1.085505,1.087548,1.087548,0
2024-05-21,1.086083,1.087548,1.084599,1.086083,1.086083,0
2024-05-22,1.085517,1.086484,1.082251,1.085517,1.085517,0


In [16]:
S_0 =  data.iloc[-1, -3]
S_0

1.0854227542877195

In [17]:
data['Log_Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))

# Drop NaN values that appear due to shift
data = data.dropna()

# Calculate the daily volatility (standard deviation of daily returns)
daily_volatility = data['Log_Returns'].std(ddof=1)

# Annualize the daily volatility
annualized_volatility = daily_volatility * np.sqrt(252)

C:\Users\saidd\AppData\Local\Temp\ipykernel_23112\3250488479.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Log_Returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))


In [18]:
annualized_volatility, daily_volatility

(0.07840915320728338, 0.004939312378515103)

In [19]:
r_d = 5.209/100
r_f = 3.67/100

# d1 and d2 for C1
d1 = (np.log(1/0.9) + (r_d - r_f + annualized_volatility**2/2))/annualized_volatility
d2 = d1 - annualized_volatility

In [20]:
r_f

0.036699999999999997

In [21]:
d1, d2

(1.5792099295323851, 1.5008007763251017)

In [22]:
# Calculate N(d1) and N(d2) for C1
N_d1 = stats.norm.cdf(d1)
N_d2 = stats.norm.cdf(d2)

print(f"N(d1) = {N_d1}")
print(f"N(d2) = {N_d2}")

N(d1) = 0.9428560434216743
N(d2) = 0.9332964510801315


In [23]:
# price of C1
S_0*np.exp(-r_f) * N_d1 - S_0*0.9*np.exp(-r_d)*N_d2

0.12107621472589358

In [24]:
# d1 and d2 for C2

d1_c2 = (np.log(1/1.1) + (r_d - r_f + annualized_volatility**2/2))/annualized_volatility
d2_c2 = d1_c2 - annualized_volatility
d1_c2, d2_c2

(-0.9800664719313035, -1.058475625138587)

In [25]:
# Calculate N(d1) and N(d2) for C2

N_d1_c2 = stats.norm.cdf(d1_c2)
N_d2_c2 = stats.norm.cdf(d2_c2)

print(f"N(d1) = {N_d1_c2}")
print(f"N(d2) = {N_d2_c2}")

N(d1) = 0.16352665395855154
N(d2) = 0.14491932881484376


In [26]:
# price of C2
S_0*np.exp(-r_f) * N_d1_c2 - S_0*1.1*np.exp(-r_d)*N_d2_c2

0.006853276798729907

In [45]:
# delta C1
delta_c1 = np.exp(-r_f)*N_d1
delta_c1

0.9088804913669761

In [46]:
# delta C2
delta_c2 = np.exp(-r_f)*N_d1_c2
delta_c2

0.15763401702562516

In [41]:
# gamma of C1
gamma_C1= np.exp(-r_f) * (1/(2*pi)**0.5) * np.exp(-d1**2/2)*(1/annualized_volatility)*(1/S_0)
gamma_C1

1.298558142759702

In [42]:
# gamma of C2
gamma_C2= np.exp(-r_f) * (1/(2*pi)**0.5) * np.exp(-d1_c2**2/2)*(1/annualized_volatility)*(1/S_0)
gamma_C2

2.795305447137758

In [44]:
k = gamma_C2/gamma_C1
k

2.1526224780333374

In [52]:
n_2_w = 1/(k+1-k*delta_c1 + delta_c2)
n_2_w

0.738672501784468

In [53]:
n_1_w = n_2_w*k
n_1_w

1.5900830312463663

In [54]:
n_3_w = (k*delta_c1-delta_c2)*(-n_2_w)
n_3_w

-1.3287555330308343

In [56]:
n_1_w+n_2_w + n_3_w

1.0

In [59]:
CAPITAL = 100000

-125503.94949999999

In [63]:
# Expected return for portfolio 

CAPITAL*np.exp(0.05209)

105347.05505964908

In [67]:
# Theta for C1
theta_1 = 0.5*S_0*np.exp(-r_f)*(1/(2*pi)**0.5)*np.exp(-d1**2/2) * annualized_volatility * 1
theta_1 -= r_f*S_0*np.exp(-r_f)*N_d1
theta_1 += r_d*S_0*0.9*np.exp(-r_d)*N_d2
theta_1

0.01357854394499649

In [68]:
# Theta for C2
theta_2 = 0.5*S_0*np.exp(-r_f)*(1/(2*pi)**0.5)*np.exp(-d1_c2**2/2) * annualized_volatility * 1
theta_2 -= r_f*S_0*np.exp(-r_f)*N_d1_c2
theta_2 += r_d*S_0*1.1*np.exp(-r_d)*N_d2_c2
theta_2

0.012399733904717276

In [69]:
# theta of portfolio

N1 = 126689.292
N2 = 58826.85804

N1*theta_1 - N2*theta_2

990.8187326359141